# Mechanical MNIST Cahn-Hilliard

In [1]:
# import required packages
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers, models, activations, callbacks#metrics, callbacks, regularizers
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from pathlib import Path

## Metamodel
**Network Structure:** A Convolutional Neural Network (with 9 hidden layers) is used as a metamodel for single quantity of interest prediction (strain energy).  
**Input:** 64x64 images of material distribution (cahn-hilliard patterns)  
**Output:** strain energy values

In [2]:
# create network structure in tensorflow
def create_model_cahnhill():
    # create a simple CNN model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation=None, input_shape=(64, 64, 1)))
    model.add(layers.BatchNormalization(axis=1))
    model.add(layers.Activation(activations.relu))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation=None)) #input: 32x32 with 32 channels
    model.add(layers.BatchNormalization(axis=1))
    model.add(layers.Activation(activations.relu))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation=None)) #input: 16x16 with 64 channels
    model.add(layers.BatchNormalization(axis=1))
    model.add(layers.Activation(activations.relu))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), padding='valid', activation=None)) #input: 8x8 with 128 channels
    model.add(layers.BatchNormalization(axis=1))
    model.add(layers.Activation(activations.relu))
    model.add(layers.Conv2D(256, (3, 3), padding='valid', activation=None)) #input: 6x6 with 256 channels
    model.add(layers.BatchNormalization(axis=1))
    model.add(layers.Activation(activations.relu))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(512, (4, 4), padding='valid', activation=None)) #input: 4x4 with 256 channels
    model.add(layers.BatchNormalization(axis=1))
    model.add(layers.Activation(activations.relu))
    model.add(layers.Conv2D(512, (1, 1), padding='valid', activation=None))
    model.add(layers.BatchNormalization(axis=1))
    model.add(layers.Activation(activations.relu))
    model.add(layers.Conv2D(128, (1, 1), padding='valid', activation=None))
    model.add(layers.BatchNormalization(axis=1))
    model.add(layers.Activation(activations.relu))
    model.add(layers.Conv2D(1, (1, 1), padding='valid', activation=None))
    model.add(layers.Flatten())

    return model

To see a model summary run this cell

In [ ]:
model = create_model_cahnhill()
model.summary()

## Datasets and Training Procedure
**Core Dataset:** A fixed subset of the real Cahn Hilliard patterns with 1000 samples  
**Generated Patterns:** 59000 patterns generated using the StyleGAN network trained on the **Core** dataset  
**Extra Real Patterns:** 59000 patterns to extend the size of training set and 12000 patterns for the test set  

We use a **5-fold cross-vallidation** to evaluate the metamodel performance.

**Method:**
The core dataset is partitioned into 5 subsamples each containing 200 samples. One subsample is choosen as the validation set to choose the best model during the training process and the other 4 subsamples are used as training data and the process will be repeated 5 times with different validation sets.  
The training set size can be extended with **Real** or **Generated** patterns. We add $k = \{0, 1000, 3000, 7000, 15000, 31000, 59000\}$ extra patterns to the training set. In addition, rotation $\{90^\circ, 180^\circ, 270^\circ\}$ can be used to augment the training set.

**Note**  
Strain energy values for low fidelity simulation are in the order of $10^{-6}$ since only perturbation analysis is considered. This causes some problem for the convergence of network parameters. Therefore, we multiply output values by $10^6$.

In [3]:
# loading the dataset
core_dataset = [np.load('lowfi-inputs.npy').reshape(-1,64,64,1)[0:1000], 1e6*np.load('lowfi-strain-energy.npy')[0:1000]]
extra_real = [np.load('lowfi-inputs.npy').reshape(-1,64,64,1)[1000:60000], 1e6*np.load('lowfi-strain-energy.npy')[1000:60000]]
extra_generated = [np.load('generated-scc.npy').reshape(-1,64,64,1), 1e6*np.load('generated-scc-energy.npy')]
test_dataset = [np.load('lowfi-inputs.npy').reshape(-1,64,64,1)[60000:], 1e6*np.load('lowfi-strain-energy.npy')[60000:]]

In [4]:
def training(core_data, extra_data, k=0, rot_aug_flag=False, extra_data_type=''):
    for fold in range(5):
        res_folder = 'results'
        ########## partitioning the core dataset ##########
        # inputs
        a1, a2, a3 = np.split(core_data[0], [fold*200, (fold+1)*200])
        train_images = np.vstack((a1, a3))
        validation_images = a2

        # labels
        a1, a2, a3 = np.split(core_data[1], [fold*200, (fold+1)*200])
        train_labels = np.hstack((a1, a3))
        validation_labels = a2

        ########## adding extra samples to the training set ##########
        if k:
            perm = np.random.permutation(800+k)
            train_images = np.vstack((train_images, extra_data[0][0:k]))[perm]
            train_labels = np.hstack((train_labels, extra_data[1][0:k]))[perm]
            res_folder = res_folder+f'-{extra_data_type}-{k}'

        ########## training set augmentation with rotation ##########
        if rot_aug_flag:
            train_images = np.vstack((train_images, np.rot90(train_images, 1, [1,2]), np.rot90(train_images, 2, [1,2]), np.rot90(train_images, 3, [1,2])))
            train_labels = np.hstack((train_labels, train_labels, train_labels, train_labels))
            res_folder = res_folder+f'-wAug'

        ########## Training Process ##########
        Path(res_folder).mkdir(parents=True, exist_ok=True)

        optim = tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)

        # save best model
        mcp_save = callbacks.ModelCheckpoint(filepath=res_folder+f'/model-'+str(fold)+'.h5', monitor='val_loss', mode='min', save_best_only=True)
        
        # save training and validation error at each epoch
        csv_logger = callbacks.CSVLogger(res_folder+f'/results-{fold}.csv')
        
        my_callbacks = [csv_logger, mcp_save]

        model = create_model_cahnhill()
        model.compile(optimizer=optim,loss='mse')

        history = model.fit(train_images, train_labels, epochs=200, batch_size=64, callbacks=my_callbacks, validation_data=(validation_images, validation_labels))
        
def evaluate(data, model_name):
    model = create_model_cahnhill()
    model.compile(loss='mse')
    
    print(model_name)
    model.load_weights(model_name)
    mse = model.evaluate(data[0], data[1], batch_size=64)
    pred = model.predict(data[0])
    coeffs = np.polyfit(pred.reshape(-1), data[1].reshape(-1), 1)
    p = np.poly1d(coeffs)
    pp = p(pred)
    r2 = r2_score(pp, data[1])
    
    return mse, r2

In [ ]:
# training a model on the core dataset with 1000 extra generated samples without rotational data augmentation
training(core_dataset, extra_generated, k=1000, rot_aug_flag=False, extra_data_type='generated')

In [ ]:
# evaluating model performance on the test set
mse = np.zeros(5)
r2 = np.zeros(5)
for fold in range(5):
    mse[fold], r2[fold] = evaluate(test_dataset, f'results-generated-1000/model-{fold}.h5')

print('\nAverage R2 Score of the trained models:', np.mean(r2))

# Cahn-Hilliard Pattern Synthesis
We used [StyleGAN2 with Adaptive Discriminator Augmentation](https://github.com/NVlabs/stylegan2-ada-pytorch) for patterns synthesis with a generative model. To install all requirements and prerequisites of for library please follow the instruction in [stylegan2-ada-reqs](https://github.com/NVlabs/stylegan2-ada-pytorch#requirements).  

## Dataset Preparation
We use 1000 samples in the core dataset to train a generative model. To prepare the dataset for training, we need a folder containing images of Cahn-Hilliard patterns.

In [ ]:
import cv2